# Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

In [2]:
model_name = 'multi-qa-distilbert-cos-v1'
model = SentenceTransformer(model_name)

In [3]:
user_question = "I just discovered the course. Can I still join it?"
q1= model.encode(user_question)

In [4]:
print(q1[0])

0.07822259


# Prepare the documents

In [5]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [6]:
documents = [item for item in documents if item['course'] == 'machine-learning-zoomcamp']

In [7]:
len(documents)

375

# Q2. Creating the embeddings

In [8]:
from tqdm import tqdm
import numpy as np

In [9]:
embeddings = []
for doc in tqdm(documents):
    qa_text = f"{doc['question']} {doc['text']}"
    embedding = model.encode(qa_text)
    embeddings.append(embedding)

X = np.array(embeddings)
print(X.shape)

100%|█████████████████████████████████████████| 375/375 [00:44<00:00,  8.47it/s]

(375, 768)


# Q3. Search

In [10]:
scores = X.dot(q1)
print(np.max(scores))

0.6506574


# Vector search

In [11]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(q1, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

# Q4. Hit-rate for our search engine

In [12]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [13]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    v_doc_question = model.encode(q['question'])
    results = search_engine.search(v_doc_question, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|███████████████████████████████████████| 1830/1830 [00:25<00:00, 70.57it/s]


In [14]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [15]:
hit_rate(relevance_total)

0.9398907103825137

# Q5. Indexing with Elasticsearch

In [16]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [17]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_text_vector'] = model.encode(qt).tolist()

    es_client.index(index=index_name, id=doc['id'], body=doc)

100%|█████████████████████████████████████████| 375/375 [00:34<00:00, 10.86it/s]


In [18]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    result_docs = es_client.search(
        index=index_name,
        body=search_query
    )

    return result_docs['hits']['hits']

In [19]:
user_question = "I just discovered the course. Can I still join it?"
q5 = model.encode(user_question).tolist()
response = elastic_search_knn("question_text_vector", q5, "machine-learning-zoomcamp")

for metric in response:
    print(f"Score: {metric['_score']}, ID: {metric['_id']}, Question: {metric['_source']['question']}")

Score: 0.8253288, ID: ee58a693, Question: The course has already started. Can I still join it?
Score: 0.7358538, ID: 0a278fb2, Question: I just joined. What should I do next? How can I access course materials?
Score: 0.7295, ID: 6ba259b1, Question: I filled the form, but haven't received a confirmation email. Is it normal?
Score: 0.72849524, ID: 9f261648, Question: Can I do the course in other languages, like R or Scala?
Score: 0.7252793, ID: e7ba6b8a, Question: The course videos are from the previous iteration. Will you release new ones or we’ll use the videos from 2021?


# Q6. Hit-rate for Elasticsearch

In [20]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [21]:
def vector_question_text_elastic(q):
    question = q['question']
    
    v_q = model.encode(question)

    return elastic_search_knn("question_text_vector", v_q, "machine-learning-zoomcamp")

In [22]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return hit_rate(relevance_total)

In [23]:
evaluate(ground_truth, vector_question_text_elastic)

100%|███████████████████████████████████████| 1830/1830 [04:06<00:00,  7.43it/s]


0.9398907103825137